Here we implement the solver from [Quantum algorithm for solving linear differential equations: Theory and experiment](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307) by Xin *et al.* in the **Classiq** quantum programming language. 

### Classical Problem

Xin *et al.* provide a new method to solve any Linear Differential Equation formulated as 

$$\frac{d}{dt}\vec{x}(t)= M\vec{x}(t)+\vec{b}$$

for an $N\times N$ matrix $M$ and $N$-dimensional vectors $x$, $b$. 

Taylor-expanding the analytical solution, 

$$
\begin{aligned}
x(t)&=e^{Mt}x(0)+\left(e^{Mt}-\mathbb{I}\right)M^{-1}b
\\
&\approx \sum^k_{m=0}\frac{(Mt)^m}{m!}x(0)+\sum^k_{n=1}\frac{M^{n-1}t^n}{n!}b.
\end{aligned}
$$

### Quantum Formulation

1. Prepare two quantum states $\ket{x(0)}$ and $\ket{b}$ in the $N$-dimensional computational basis, on $\log_2(N)$ qubits. 
1. Describe the $N\times N$ operator $M$ as $A$ in the computational basis, such that $A_{ij}=M_{ij}\ket{i}\bra{j}$ up to normalization.

We then arrive at 

$$\ket{x(t)}\approx \sum_{m=0}^k \frac{\|x(0)\|(\|M\|At)^m}{m!}\ket{x(0)}+\sum_{n=1}^k\frac{\|b\|(\|M\|A)^{n-1}t^n}{n!}\ket{b}$$


#### More Encodings
1. Assuming $A$ is unitary, encode $k$ powers of $A$ as the unitaries $U_n=A^n$.
1. Define $C_m=\|x(0)\|(\|M\|t)^m/m!$ and $D_n=\|b\|(\|M\|t)^{n-1}t/n!$



### Using Classiq

You need to install the `classiq` Python module from `pip` beforehand. You can do this with Anaconda via the provided `.yml` file with the terminal line `conda env create -f environment.yml`. 

You then need to authenticate the module by running the following code block once, which will pop up a login window in your browser. 

In [3]:
# ONLY RUN THIS ONCE TO AUTHENTICATE DEVICE
import classiq
classiq.authenticate(overwrite=True)

/opt/miniconda3/envs/classiq-env/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:92: UserWarning: Overwriting an existing refresh token should only be done if it is compromised. Make sure this operation is necessary, and if not, remove the call to device registration.
  warnings.warn(


Your user code: HBSM-KJFC
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=HBSM-KJFC


First, we import all relevant modules and Classiq functions, as well as handy utility functions we wrote in `util.py`.

In [3]:
import numpy as np
np.set_printoptions(precision=3)
import math
import os

from classiq import (
    qfunc, Output, QBit, QNum, QArray, CInt,
    allocate, within_apply, control, repeat, if_, 
    #inplace_prepare_state, hadamard_transform, inplace_prepare_int, 
    inplace_prepare_amplitudes,
    create_model, write_qmod,
    X, H, SWAP, I,
    unitary,
    Constraints, Preferences, 
)

from util import quantum_encode, is_unitary, get_work_strings, quantum_vectorize, run_qmod

# wierd bug with asyncio and jupyter notebooks...
import nest_asyncio
nest_asyncio.apply()

To submit the code to Classiq and process it, we wrap all of the native `qfunc`s with a larger function that allows us to vary the order of the Taylor approximation $k$, the time $t$, and the error bound of the amplitude encoding.

In [4]:
class QuantumLDESolver():
    """
    Class of quantum algorithms implemented from "Quantum algorithm for solving 
     linear differential equations: Theory and experiment" by Xin et. al. (2020) 
     at https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307

     TODO: implement non-unitary M solving
    """

    def __init__(self, M, x0, b, k=2, err_bound=0.01, 
                 print_info=True):
        """ Initialize a single LDE problem of dx/dt = Mx + b
        Inputs:
         -  M: matrix of the LDE
         - x0: initial condition vector
         -  b: forcing vector
         -  k: order of Taylor expansion
         - err_bound: error bound for Classiq's coefficient approximation
        """
        self.k = k
        self.err_bound = err_bound
        self.cwd = os.getcwd()
        self.data = None
        
        self.x0_norm, self.x0 = quantum_encode(x0)
        self.b_norm = np.linalg.norm(b)
        self.b = b
        if self.b_norm != 0:
            self.b_norm, self.b = quantum_encode(b)
        
        if print_info:
            print('x0',self.x0,self.x0_norm)
            print('b',self.b, self.b_norm)
        
        # use order-2 norm to preserve unitarity
        self.M_norm = np.linalg.norm(M, ord=2)
        A = M / self.M_norm
        self.isUnitary = is_unitary(A)
        if not self.isUnitary:
            raise NotImplementedError("Non-unitary M not yet implemented")
        self.U = [np.linalg.matrix_power(A, i).tolist() for i in range(0,k+1)]
        
        self.nwork = math.ceil(math.log2(len(self.x0)))
        self.ntaylor = math.ceil(math.log2(self.k+1))
        nqubits = self.nwork + self.ntaylor
        nqubits += 1 if self.b_norm != 0 else 0
        if print_info:
            print(f'Needs at least {nqubits} qubits to run!')

        if print_info:
            print(f"Initialized LDE solver with k={k}, err_bound={err_bound}")
            print(f"Initial condition x0={x0} with norm {self.x0_norm}")
            print(f"Matrix M={M} with norm {self.M_norm}")
            print(f"Forcing vector b={b} with norm {self.b_norm}")
            print(f"Matrix A={A} is {'unitary' if self.isUnitary else 'not unitary'}")
    
    def get_Cs(self, M_norm, x0_norm, k, t):
        ''' returns (C_norm, C_amps) when b is zero '''
        C = np.zeros(k+1)
        C[0] = x0_norm
        for i in range(1,k+1):
            C[i] = C[i-1] * M_norm * t / i
        # encode amplitudes as probabilities
        Cnorm, C_amps = quantum_vectorize(C)
        return Cnorm, C_amps.tolist()
    
    def inp_multi_reg_amps(self, *args):
        """ prepare multiple QArray amplitudes at once
        Inputs: register1, amps1, register2, amps2, ... registerN, ampsN
        """
        for i in range(0, len(args), 2):
            inplace_prepare_amplitudes(amplitudes=args[i+1], 
                                       target=args[i], bound=self.err_bound)

    def LDE_no_time_qmod(self, save_qmod=''):
        """
        t = 0 case, trivially return x0, still in qmod for consistent output
        """
        @qfunc
        def main(work: Output[QArray], taylor: Output[QNum]):
            # allocate all qubits before doing anything
            allocate(self.nwork, work)
            allocate(self.ntaylor, taylor)
            # encode x0
            self.inp_multi_reg_amps(work, self.x0)
        
        qmod = create_model(main)
        if save_qmod:
            write_qmod(qmod, save_qmod)
        return qmod

    def unitary_LDE_nob_qmod(self, C_amps, save_qmod=''):
        """ 
        M is unitary AND b = 0 
        """
        @qfunc
        def prepare_registers(work: QArray, taylor: QNum):
            # evolve ancilla into superposition state
            # encode x_0 and Cvals into registers with |0> ancilla
            self.inp_multi_reg_amps(work, self.x0, taylor, C_amps)

        @qfunc
        def do_entangling(work: QArray, taylor: QNum):
            # apply powers of A to taylor register
            for i in range(self.k+1):
                control(taylor == i,
                        lambda: unitary(self.U[i], work))

        @qfunc
        def main(work: Output[QArray], taylor: Output[QNum]):
            # allocate all qubits before doing anything
            allocate(self.nwork, work)
            allocate(self.ntaylor, taylor)
            # apply V^dag U V
            within_apply(lambda: prepare_registers(work, taylor),
                         lambda: do_entangling(work, taylor))
        
        qmod = create_model(main)
        if save_qmod:
            write_qmod(qmod, save_qmod)
        return qmod

    def solve(self, times: list[float], opt='depth', nshots=10000, sim='Classiq', 
              open_circuit=False, print_info=True):
        # write directory
        dirpath = f"{os.getcwd()}/runs/k={self.k}/err_bound={self.err_bound}"
        if not os.path.isdir(dirpath):
            os.makedirs(dirpath+'/qmod')
            os.makedirs(dirpath+'/qprog')
        
        allData = []
        for t in times:
            # name file
            name = 'LDE'
            if self.isUnitary: name += "_U"
            if self.b_norm == 0: name += '_no-b'
            name += f'_t={round(t,3)}'
            # generate .qmod file 
            if t == 0:
                qmod = self.LDE_no_time_qmod(f'{dirpath}/qmod/{name}')
                #### !!!!!! ####
                #### !!!!!! ####
                N = np.sqrt(self.x0_norm)
                #### !!!!!! ####
                #### !!!!!! ####
            else:
                N, C_amps = self.get_Cs(self.M_norm, self.x0_norm, self.k, t)
                qmod = self.unitary_LDE_nob_qmod(C_amps, f'{dirpath}/qmod/{name}')
            # run the quantum circuit on backend
            ret = run_qmod(qmod, opt, nshots, name, f'{dirpath}/qprog/{name}', sim, open_circuit, print_info)
            allData.append((N,ret))
        self.data = allData
        return allData
    
    def parse_single(self, N, rdict, print_info=False):
        """
        returns a dictionary of counts from classiq result package
        - N: normalization factor to reconstruct vector with
        - rdict: result dictionary from classiq
        """
        x0 = self.x0
        y = np.zeros((len(x0),))
        total_counts = rdict.num_shots
        # reconstruct / parse state vector
        out_map = rdict.output_qubits_map   
        vec_indices = out_map['work']         # tuple of indices
        assert 2**len(vec_indices) == len(x0)
        lsb_right = rdict.counts_lsb_right  # whether output map starts from right or left of string
        counts_dict = rdict.counts
        # set all other qubits to be 0, then reconstruct state vector from counts
        nqubits = len(list(counts_dict.keys())[0])
        # only care about all ancillas being zero
        work_strings = get_work_strings(nqubits, vec_indices, lsb_right)
        # iterate from least-significant to most-significant bitstring
        for j, work_string in enumerate(work_strings):
            if print_info:
                print(f'parsing |{work_string}> as x[{j}]')
            if work_string not in counts_dict:
                print(f"Work string {work_string} not found in counts!")
                continue
            # convert to probabilities = amp^2
            y[j] = counts_dict[work_string] / total_counts
        # convert to amplitudes and multiply by total normalization
        #### !!!!!! ####
        #### !!!!!! ####
        y = np.sqrt(y) * N**2
        #### !!!!!! ####
        #### !!!!!! ####
        #### !!!!!! ####
        return y
    
    def parse_data(self, data=None, print_info=True):
        """ 
        returns len(times) x len(x0) numpy array of generated solution
        """
        if data is None:
            if self.data is None:
                raise ValueError('run solve_times first to generate data, or input your own data')
            else:
                data = self.data
        x0 = self.x0
        y = np.zeros((len(data), len(x0)))
        for i in range(len(data)):
            N = data[i][0]    # normalization amount
            rdict = data[i][1]  # classiq results dictionary
            total_counts = rdict.num_shots
            
            # reconstruct / parse state vector
            out_map = rdict.output_qubits_map   
            vec_indices = out_map['work']         # tuple of indices
            assert 2**len(vec_indices) == len(x0)
            lsb_right = rdict.counts_lsb_right  # whether output map starts from right or left of string
            counts_dict = rdict.counts
            # set all other qubits to be 0, then reconstruct state vector from counts
            nqubits = len(list(counts_dict.keys())[0])
            # only care about all ancillas being zero
            work_strings = get_work_strings(nqubits, vec_indices, lsb_right)
            
            # iterate from least-significant to most-significant bitstring
            for j, work_string in enumerate(work_strings):
                if print_info:
                    print(f'parsing |{work_string}> as x[{j}]')
                if work_string not in counts_dict:
                    print(f"Work string {work_string} not found in counts!")
                    continue
                # convert to probabilities = amp^2
                y[i,j] = counts_dict[work_string] / total_counts
            # convert to amplitudes and multiply by total normalization
            #### !!!!!! ####
            #### !!!!!! ####
            #### !!!!!! ####
            y[i,:] = np.sqrt(y[i,:]) * N**2
            #### !!!!!! ####
            #### !!!!!! ####
            #### !!!!!! ####
        return y

### Our Problem
We solve the following differential equation:
$$\frac{d^2y(t)}{dt^2}=y''(t)=-\omega^2 y'(t),\quad y(0)=y'(0)=1\,.$$

We can linearize this in the form $\frac{d}{dt}\mathbf{x} = M\mathbf{x}+\mathbf{b}$ using $x=[y(t),y'(t)]$ to get

$$\frac{d}{dt}\begin{pmatrix}y\\ y'\end{pmatrix} = \begin{bmatrix}0 & 1 \\ -\omega^2 & 0\end{bmatrix}\begin{pmatrix}y\\ y'\end{pmatrix} + \begin{bmatrix}0 \\ 0\end{bmatrix}\,.$$

When $\omega=1$, $M$ is unitary, which allows us to use the easy version of the algorithm that doesn't rely on accessing $L$-level qudits. Furthermore, $b=0$, which means we can ignore the first ancilla qubit and only have to compute half of the Taylor expansion coefficients.

Let's Taylor-expand our solution to 2nd-order and solve for 11 points over $t\in[0,1]$. 

In [7]:
omega = 1
M = np.array([[0,1],[-omega**2,0]])
x0 = [1,1]
b = [0,0]

k = 3
err_bound = 0.01
solver = QuantumLDESolver(M, x0, b, k, err_bound)

num_timesteps = 8
time_range = (0,1)
times = np.linspace(time_range[0],time_range[1],num_timesteps)

results = solver.solve(times, opt='depth', nshots=1000, sim='Classiq', open_circuit=False)
yt = solver.parse_data(results)

x0 [0.7071067811865475, 0.7071067811865475] 1.4142135623730951
b [0, 0] 0.0
Needs at least 3 qubits to run!
Initialized LDE solver with k=3, err_bound=0.01
Initial condition x0=[1, 1] with norm 1.4142135623730951
Matrix M=[[ 0  1]
 [-1  0]] with norm 1.0
Forcing vector b=[0, 0] with norm 0.0
Matrix A=[[ 0.  1.]
 [-1.  0.]] is unitary
Running LDE_U_no-b_t=0.0
	circuit synthesized in 1.95s: width=3,depth=1
	job with 1000 shots is QUEUED on provider-backend=Classiq-simulator 
	and can be accessed at https://platform.classiq.io/jobs/12fc9197-9b05-4b0d-9d07-a8d896f29800
	ran in 2.89s
Running LDE_U_no-b_t=0.143


ClassiqAPIError: Error number 229004 occurred. Please try again later.

Error identifier: E0217297C-08B9-4E98-A785-9E08CB7FE2B6.
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack

In [6]:
# moved to update the parse_data function
yt = QuantumLDESolver(M, x0, b, k, err_bound).parse_data(results)
#yt = solver.parse_data(results)
import matplotlib.pyplot as plt
# use absolute value since we can only recover absolute amplitudes,
# no phase recovery!
def analytical_soln(w, times):
    # x, x'
    x_xdiv = [np.cos(w*times) + np.sin(w*times)/w, -w*np.sin(w*times) + np.cos(w*times)]
    return np.abs(np.array(x_xdiv)).T

smooth_times = np.linspace(times[0],times[-1],50)
plt.plot(smooth_times, analytical_soln(1, smooth_times), '--',label=['$x_{th}(t)$','$x_{th}\'(t)$'])
print(times.shape, yt.shape)
plt.scatter(times, yt[:,0], label='$x(t)$', marker='+')
plt.scatter(times, yt[:,1], label="$x'(t)$", marker='x')
plt.xlabel('time (s)')
plt.ylabel(r'$|\vec{x}(t)|$')
plt.title('Quantum LDE Solver')
plt.legend()

x0 [0.7071067811865475, 0.7071067811865475] 1.4142135623730951
b [0, 0] 0.0
Needs at least 3 qubits to run!
Initialized LDE solver with k=3, err_bound=0.01
Initial condition x0=[1, 1] with norm 1.4142135623730951
Matrix M=[[ 0  1]
 [-1  0]] with norm 1.0
Forcing vector b=[0, 0] with norm 0.0
Matrix A=[[ 0.  1.]
 [-1.  0.]] is unitary


NameError: name 'results' is not defined

Now, let's print the results to make sure everything is working as intended!

In [ ]:
m = 1
#w = np.sqrt(k/m)
# m = k/w^2
k = m/(omega**2)
def KE(y):
    # kinetic energy = 1/2 m (y')^2
    return np.sum(y[:,1]**2)/2

def PE(y):
    # 1/2 k y^2
    return k*np.sum(y[:,0]**2)/2
